In [1]:
import tensorflow as tf
from tensorflow import keras

### When to Use Sequnetial Model
- Keras has two types of models Sequential and API.
- Sequential model is a stack of plain layers
- If there is no need of connecting one layer with multiple layers sequential model is the best one
- A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.

In [2]:
# Define a sequential model with 3 layers
model = keras.Sequential(
    [
    keras.layers.Dense(2, activation="relu", name="layer1"),
    keras.layers.Dense(3, activation="relu", name="layer2"),
    keras.layers.Dense(4, name="layer3")
    ])
# Call model on a test input
x = tf.ones([3,3])
y = model(x)

In [4]:
# which is equivalent to following code
layer1 = keras.layers.Dense(2, activation="relu", name="layer1")
layer2 = keras.layers.Dense(3, activation="relu", name="layer2")
layer3 = keras.layers.Dense(4, name="layer3")

# lets test this model again
x = tf.ones([3,3])
y = layer3(layer2(layer1(x)))

### A sequential Model is not appropriate when
- when a layers should connect to multiple inputs or multiple outputs
- Overall model has multiple inputs or multiple outputs
- When there is a need of layers sharing
- When we want to develop a non-linear topology (e.g., a residula connection, a multi-branch model)

In [5]:
# we can create a Sequential model by passing a list of layers to the Sequential constructor:
model = keras.Sequential(
    [
        keras.layers.Dense(2, activation="relu"),
        keras.layers.Dense(3, activation="relu"),
        keras.layers.Dense(4),
    ]
)
# Its layers are accessible via the layers attribute:
model.layers

In [6]:
# You can also create a Sequential model incrementally via the add() method:
model = keras.Sequential()
model.add(keras.layers.Dense(2, activation="relu"))
model.add(keras.layers.Dense(3, activation="relu"))
model.add(keras.layers.Dense(4))

# Note that there's also a corresponding pop() method to remove layers: 
# a Sequential model behaves very much like a list of layers.
model.pop()
print(len(model.layers))  # 2

2


In [9]:
# Also note that the Sequential constructor accepts a name argument, just like any layer or model in Keras. 
# This is useful to annotate TensorBoard graphs with semantically meaningful names.


model = keras.Sequential(name="my_sequential")
model.add(keras.layers.Dense(2, activation="relu", name="layer1"))
model.add(keras.layers.Dense(3, activation="relu", name="layer2"))
model.add(keras.layers.Dense(4, name="layer3"))
model.layers


### Specifying the input shape in Advance
- Generally, all layers in Keras need to know the shape of their inputs in order to be able to create their weights. 
- So when you create a layer like this, initially, it has no weights:

In [11]:
layer = keras.layers.Dense(3)
layer.weights  # Empty

[]

In [12]:
# It creates its weights the first time it is called on an input, 
# since the shape of the weights depends on the shape of the inputs:
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

[<tf.Variable 'dense_6/kernel:0' shape=(4, 3) dtype=float32, numpy=
 array([[-0.37210792, -0.1146307 , -0.22423726],
        [-0.2684635 ,  0.16397238, -0.3094741 ],
        [-0.7576456 , -0.6382345 ,  0.7912996 ],
        [-0.6900034 ,  0.46272528, -0.28667063]], dtype=float32)>,
 <tf.Variable 'dense_6/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [13]:
# Naturally, this also applies to Sequential models. 
# When you instantiate a Sequential model without an input shape, it isn't "built": it has no weights (and calling model.weights results in an error stating just this).
# The weights are created when the model first sees some input data:
model = keras.Sequential(
    [
        keras.layers.Dense(2, activation="relu"),
        keras.layers.Dense(3, activation="relu"),
        keras.layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)
print("Number of weights after calling the model:", len(model.weights))  # 6

Number of weights after calling the model: 6


In [14]:
# Once a model is "built", you can call its summary() method to display its contents:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (1, 2)                    10        
                                                                 
 dense_8 (Dense)             (1, 3)                    9         
                                                                 
 dense_9 (Dense)             (1, 4)                    16        
                                                                 
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [16]:
# However, it can be very useful when building a Sequential model incrementally to be able to display the summary of the model so far, including the current output shape. 
# In this case, you should start your model by passing an Input object to your model, so that it knows its input shape from the start:
model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(keras.layers.Dense(2, activation="relu"))

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Note that the Input object is not displayed as part of model.layers, since it isn't a layer:
model.layers

In [20]:
# A simple alternative is to just pass an input_shape argument to your first layer:
model = keras.Sequential()
model.add(keras.layers.Dense(2, activation="relu", input_shape=(4,)))

model.summary()

# Models built with a predefined input shape like this always have weights (even before seeing any data) and always have a defined output shape.

# In general, it's a recommended best practice to always specify the input shape of a Sequential model in advance if you know what it is.

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 2)                 10        
                                                                 
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


### A common debugging workflow: add() + summary()
When building a new Sequential architecture, it's useful to incrementally stack layers with add() and frequently print model summaries. For instance, this enables you to monitor how a stack of Conv2D and MaxPooling2D layers is downsampling image feature maps:

In [22]:
model = keras.Sequential()
model.add(keras.Input(shape=(250, 250, 3)))  # 250x250 RGB images
model.add(keras.layers.Conv2D(32, 5, strides=2, activation="relu"))
model.add(keras.layers.Conv2D(32, 3, activation="relu"))
model.add(keras.layers.MaxPooling2D(3))

# Can you guess what the current output shape is at this point? Probably not.
# Let's just print it:
model.summary()

# The answer was: (40, 40, 32), so we can keep downsampling...

model.add(keras.layers.Conv2D(32, 3, activation="relu"))
model.add(keras.layers.Conv2D(32, 3, activation="relu"))
model.add(keras.layers.MaxPooling2D(3))
model.add(keras.layers.Conv2D(32, 3, activation="relu"))
model.add(keras.layers.Conv2D(32, 3, activation="relu"))
model.add(keras.layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(keras.layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(keras.layers.Dense(10))

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 121, 121, 32)      9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 40, 40, 32)       0         
 )                                                               
                                                                 
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 32)      2432      
                          

### Feature Extraction with Sequential Models
Once a Sequential model has been built, it behaves like a Functional API model. This means that every layer has an input and output attribute. These attributes can be used to do neat things, like quickly creating a model that extracts the outputs of all intermediate layers in a Sequential model:

In [24]:
initial_model = keras.Sequential(
    [
    keras.Input(shape=(250, 250, 3)),
    keras.layers.Conv2D(32, 5, strides=2, activation="relu"),
    keras.layers.Conv2D(32, 3, activation="relu"),
    keras.layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers]
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

In [25]:
# Here's a similar example that only extract features from one layer:
initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        keras.layers.Conv2D(32, 5, strides=2, activation="relu"),
        keras.layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        keras.layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)

### Transfer learning with a Sequential model
- Transfer learning consists of freezing the bottom layers in a model and only training the top layers. 
- If you aren't familiar with it, make sure to read our guide to transfer learning.
- Here are two common transfer learning blueprint involving Sequential models.
- First, let's say that you have a Sequential model, and you want to freeze all layers except the last one. 
- In this case, you would simply iterate over model.layers and set layer.trainable = False on each layer, except the last one. Like this:

In [27]:
model = keras.Sequential([
    keras.Input(shape=(784)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10),
])

# Presumably you would want to first load pre-trained weights.
# model.load_weights(...)

# Freeze all layers except the last one.
for layer in model.layers[:-1]:
  layer.trainable = False

# Recompile and train (this will only update the weights of the last layer).
#model.compile(...)
#model.fit(...)

In [28]:
# Another common blueprint is to use a Sequential model to stack a pre-trained model and 
# some freshly initialized classification layers. Like this:
# Load a convolutional base with pre-trained weights
base_model = keras.applications.Xception(
    weights='imagenet',
    include_top=False,
    pooling='avg')

# Freeze the base model
base_model.trainable = False

# Use a Sequential model to add a trainable classifier on top
model = keras.Sequential([
    base_model,
    keras.layers.Dense(1000),
])

# Compile & train
#model.compile(...)
#model.fit(...)

83683744/83683744 [==============================] - 20s 0us/step
